Loading Dail_Mail Dataset

In [1]:
import pandas as pd

df=pd.read_csv('Daily_Mail.csv')
df.head()

,url,article,highlights
0,https://www.dailymail.co.uk/tvshowbiz/article-...,Beyoncé showcases her incredible figure in plu...,Beyoncé has shown off her flawless beauty in a...
1,https://www.dailymail.co.uk/tvshowbiz/article-...,Radio 1 listeners in shock as sex noises are p...,BBC Radio 1 listeners were left choking on the...
2,https://www.dailymail.co.uk/tvshowbiz/article-...,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Dan Edgar and Ella Rae Wise put on a loved-up ...
3,https://www.dailymail.co.uk/tvshowbiz/article-...,Bradley Cooper recalls 'crazy' pitch meeting a...,Bradley Cooper discussed the 'crazy' experienc...
4,https://www.dailymail.co.uk/tvshowbiz/article-...,Margaret Qualley and Beanie Feldstein stun in ...,Margaret Qualley and Beanie Feldstein were dre...


In [2]:
#using smaller dataset

df = df[['article', 'highlights']].dropna().sample(10, random_state=42).reset_index(drop=True)
df.head()

,article,highlights
0,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Bradley Cooper discussed the 'crazy' experienc...
1,The four-legged star who captured Emma Stone's...,The four-legged star who captured the heart of...
2,This Might Not Be It review: Behind the Perspe...,Beyoncé has shown off her flawless beauty in a...
3,Selena Gomez puts on a busty display in a LBD ...,"This Might Not Be It (Bush Theatre, London)"
4,Man City manager Pep Guardiola apologises to m...,Former Strictly Come Dancing star Robin 'Bobby...


Step 1 : Preprocessing

In [3]:
def clean_text(text):
    return text.strip().replace('\n', ' ')

df['clean_article'] = df['article'].apply(clean_text)
df['clean_highlights'] = df['highlights'].apply(clean_text)
df.head()

,article,highlights,clean_article,clean_highlights
0,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Bradley Cooper discussed the 'crazy' experienc...,"TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, ...",Bradley Cooper discussed the 'crazy' experienc...
1,The four-legged star who captured Emma Stone's...,The four-legged star who captured the heart of...,The four-legged star who captured Emma Stone's...,The four-legged star who captured the heart of...
2,This Might Not Be It review: Behind the Perspe...,Beyoncé has shown off her flawless beauty in a...,This Might Not Be It review: Behind the Perspe...,Beyoncé has shown off her flawless beauty in a...
3,Selena Gomez puts on a busty display in a LBD ...,"This Might Not Be It (Bush Theatre, London)",Selena Gomez puts on a busty display in a LBD ...,"This Might Not Be It (Bush Theatre, London)"
4,Man City manager Pep Guardiola apologises to m...,Former Strictly Come Dancing star Robin 'Bobby...,Man City manager Pep Guardiola apologises to m...,Former Strictly Come Dancing star Robin 'Bobby...


Step 3 : Extractive Summarization using Spacy

In [4]:
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
import spacy

nlp = spacy.load('en_core_web_sm')

def extractive_summary(text, max_sentences=2):
    doc = nlp(text)
    sentences = list(doc.sents)
    return ' '.join([str(sent) for sent in sentences[:max_sentences]])

df['extractive_summary'] = df['clean_article'].apply(lambda x: extractive_summary(x))

Step 4 : Abstractive Summarization using Transformers (BART)

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def abstractive_summary(text):
    return summarizer(text, max_length=60, min_length=20, do_sample=False)[0]['summary_text']

df['abstractive_summary'] = df['clean_article'].apply(abstractive_summary)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 60, but your input_length is only 58. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=29)


Step 5 : Evaluation

In [ ]:
for i in range(3):
    print(f"\n====== Example {i+1} ======")
    print("\nOriginal Article:\n", df['clean_article'][i])
    print("\nExtractive Summary:\n", df['extractive_summary'][i])
    print("\nAbstractive Summary:\n", df['abstractive_summary'][i])
    print("\nActual Highlights:\n", df['highlights'][i])



====== Example 1 ======

Original Article:
 TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, put on a loved-up display during trip as romance blossoms between the pair during trip to BaliBradley Cooper discussed the 'crazy' experience he had of meeting Beyonce and her husband Jay-Z while pitching the singer a movie role.

Extractive Summary:
 TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, put on a loved-up display during trip as romance blossoms between the pair during trip to BaliBradley Cooper discussed the 'crazy' experience he had of meeting Beyonce and her husband Jay-Z while pitching the singer a movie role.

Abstractive Summary:
 TOWIE's Dan Edgar, 33, and Ella Rae Wise, 23, put on a loved-up display during trip to Bali.

Actual Highlights:
 Bradley Cooper discussed the 'crazy' experience he had of meeting Beyonce and her husband Jay-Z while pitching the singer a movie role.

====== Example 2 ======

Original Article:
 The four-legged star who captured Emma Stone's heart at the BAFTAs is revealed as Lilliput the Maltese terrierThe four-legged star who captured the heart of best actress winner Emma Stone and millions of others at last night’s BAFTA Film Awards can be revealed – as Lilliput the Maltese terrier.

Extractive Summary:
 The four-legged star who captured Emma Stone's heart at the BAFTAs is revealed as Lilliput the Maltese terrierThe four-legged star who captured the heart of best actress winner Emma Stone and millions of others at last night’s BAFTA Film Awards can be revealed – as Lilliput the Maltese terrier.

Abstractive Summary:
 The four-legged star who captured the heart of best actress winner Emma Stone and millions of others at last night’s BAFTA Film Awards is revealed as Lilliput the Maltese terrier.
...
 Dan Edgar and Ella Rae Wise put on a loved-up display during a sun-kissed trip to Bali ahead of the new series.

Actual Highlights:
 Dan Edgar and Ella Rae Wise put on a loved-up display during a sun-kissed trip to Bali ahead of the new series.
Output is truncated. View as a scrollable element or open in a text editor. Adjust cell output settings...